<a href="https://colab.research.google.com/github/sefa1471/sefa1471/blob/master/Transformers_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Eğer modeli eğitecekseniz runtime/change runtime type'tan GPU'nun seçili olduğuna emin olun. Yoksa çok uzun sürebilir.

Kullanılanacak kütüphaneleri ekleyelim.
- Transformers, hazır modeller içeren ve modelleri fine-tune etmeyi kolaylaştıran bir kütüphane
- PyTorch Transformer'ın arkaplanda modeli eğitmek için kullandığı machine learning kütüphanesi
- ipywidgets'ı da ekrana textbox, buton gibi şeyleri çizdirmekte kullanıyoruz

In [ ]:
!pip install --upgrade pip
!pip install transformers
!pip install torch
!pip install ipywidgets

     |████████████████████████████████| 1.5MB 5.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 1.9 MB 4.3 MB/s 
     |████████████████████████████████| 3.2 MB 17.9 MB/s 
     |████████████████████████████████| 883 kB 49.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893258 sha256=6bba7b6c22f3135e2c95ddb115fdef11f959f11b2c1393e820ce6b2e63583435
  Stored in directory: /root/.cache/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0
Successfully built sacremoses


Eğittiğimiz modeli google drive'da saklayacağız. Bu yüzden google drive'a erişim izni vererek dosya sistemine bağlamamız gerekiyor.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Görkem Göknar'ın türkçe vikipedi metinleriyle eğittiği gpt-2 modelini baz olarak alacağız. Böylece model hazırdı Türkçenin dil yapısıyla ilgili bir çok şeyi bilir halde gelecek.

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

tokenizer = AutoTokenizer.from_pretrained("gorkemgoknar/gpt2-small-turkish")
model = AutoModelWithLMHead.from_pretrained("gorkemgoknar/gpt2-small-turkish")
# GPT-2 en fazla 1024 tokenlik bir dizi uretebiliyor
tokenizer.model_max_length=1024 


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:970: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Bu adımı çalıştırmadan önce yandaki dosyalar kısmına eğitmek için kullanacağınız corpus'u upload etmeniz gerekiyor. İsmini direk lyrics1.txt olarak bırakabilir veya aşağıda değiştirebilirsiniz. 

Corpush direk bir düz metin dosyası. En azından 500KB civarında olmalı iyi sonuç alabilmeniz için. Genelde ne kadar büyük olursa sonuç o kadar iyi olur ama eğitim de o kadar uzun sürer. Ben parçaları aralarına boş bir satırda ~ işaret koyarak ayırdım. Bunun ne işe yaradığını üretirken göreceksiniz.

In [1]:
from transformers import AutoTokenizer

train_path = 'lyrics1.txt'
test_path = 'lyrics1.txt'

from transformers import TextDataset,DataCollatorForLanguageModeling
 
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
           tokenizer=tokenizer,
           file_path=test_path,
           block_size=128)
 
    data_collator = DataCollatorForLanguageModeling(
     tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator
 
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

ModuleNotFoundError: ignored

Burada eğitim için kullanacağımız parametreleri tanımlıyoruz.  num_train_epochs parametresini istediğiniz gibi değiştirebilirsiniz. Bu eğitim sürecinde bütün corpusu sistemin kaç kere göreceğini değiştiriyor. Ne kadar yüksek olursa eğitim o kadar uzun sürer. Genelde 30-35 epochtan sonrası artık daha iyiye gitmiyor ama benim kullandığımdan daha büyük bir corpus kullanırsanız belki daha uzun süre eğitmeniz gerekebilir. 

Diğer tüm parametreler colab'da çalıştırmaya uygun. Eğer kendi ekran kartınızda çalıştaracaksanız, ekran kartının sahip olduğu ram'e göre per_device_train_batch_size ve per_device_eval_batch_size parametrelerini değiştirmeniz gerekebilir.


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

training_args = TrainingArguments(
    output_dir="./gpt2-trap", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    #learning_rate=0.00001,
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps = 100, # Number of update steps between two evaluations.
    logging_steps = 10,
    save_steps=1000, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

NameError: ignored

Aşağıdaki hücreyi çalıştırdığınız eğitim süreci başlayacak. Periyodik olarak Training loss değerini ekrana basıyor. Bu değer düşmeyi bıraktığında artık sistem daha fazla öğrenemiyor demektir.

In [ ]:
trainer.train()

Step,Training Loss
10,3.664100
20,3.582100
30,3.739900
40,3.690100
50,3.709900
60,3.571900
70,3.685900
80,3.617300
90,3.582700
100,3.669000


TrainOutput(global_step=290, training_loss=3.6144421149944437, metrics={'train_runtime': 329.558, 'train_samples_per_second': 0.88, 'total_flos': 885453942620160, 'epoch': 5.0})

Aşağıdaki hücre eğittiğimiz modeli google drive'a kaydetmemizi sağlıyor. Daha sonra yeniden kullanmak istediğimizde direk bunu yükleyebiliriz yeniden eğitmek yerine.

In [ ]:
model.save_pretrained("drive/MyDrive/Rapgen/gpt2-trap-1epoch")

Oluşturduğumuz modeli yükleyip test edebiliriz artık. Eğer daha önce eğitip kaydettiyseniz google drive'ı bağladıktan sonra direk bu hücreden başlalayabilirsiniz.


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

tokenizer = AutoTokenizer.from_pretrained("gorkemgoknar/gpt2-small-turkish")
model = AutoModelWithLMHead.from_pretrained("drive/MyDrive/Rapgen/gpt2-trap-1epoch")
# GPT-2 en fazla 1024 tokenlik bir dizi uretebiliyor
tokenizer.model_max_length=1024 


Aşağıdaki hücreyi kullanarak da modeli test edebilirsiniz. Çalıştırdığınızda bir textbox ve buton çiziyor. Textbox'ın içine metnin nasıl başlamasını istediğiniz yazıp butona tıklayarak gerisini ürettirebilirsiniz. Ben her parçayı corpusta ~ işareti ile başlattığım için direk ~ işaretiyle üretimi başlatıyorum. 

In [ ]:
from ipywidgets import widgets
model.eval();
# input sequence

textui = widgets.Textarea(
    value='~\n',
    placeholder='Type something',
    description='String:',
    disabled=False
)

button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
display(textui)
display(button)

def handle_submit(sender):
  print("yes")
  text = textui.value
  inputs = tokenizer(text, return_tensors="pt")

  # model output using Top-k sampling text generation method
  sample_outputs = model.generate(inputs.input_ids,
                                  pad_token_id=50256,
                                  do_sample=True, 
                                  max_length=500, # put the token number you want
                                  min_length=300,
                                  top_k=40, 
                                  #top_p=0.95, 
                                  temperature=0.95,
                                  num_return_sequences=1)

  # generated sequence
  for i, sample_output in enumerate(sample_outputs):
      print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))



button.on_click(handle_submit)


# >> Generated text
#    
